# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [ ]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

In [ ]:
env = UnityEnvironment(file_name="...")

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

In [ ]:
for i in range(1, 6):                                      # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))

When finished, you can close the environment.

In [ ]:
# env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [ ]:
import torch
print("PyTorch %s" % torch.__version__)
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import sys
print("Python %s" % sys.version)
import os
import random, copy, time
from collections import namedtuple, deque

### Hyperparameters

In [ ]:
BUFFER_SIZE = int(1e6)     # replay buffer size
BATCH_SIZE = 256           # minibatch size
GAMMA = 0.99               # discount factor
TAU = 1e-3                 # for soft update of target parameters
SEED = 2                   # random seed
LR_ACTOR = 1e-4            # learning rate of the actor 
LR_CRITIC = 1e-3           # l earning rate of the critic
WEIGHT_DECAY = 0           # L2 weight decay
SIGMA = 0.1                # standard deviation for noise

In [ ]:
def hidden_init(layer):
    fan_in = layer.weight.data.size()[0]
    lim = 1.0 / np.sqrt(fan_in)
    return (-lim, lim)

In [ ]:
class Actor(nn.Module):
    """Actor (Policy) Model."""

    def __init__(self, num_agents, state_size, action_size, seed, 
                 fc1_units=400, fc2_units=300):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            seed (int): Random seed
            fc1_units (int): Number of nodes in first hidden layer
            fc2_units (int): Number of nodes in second hidden layer
        """
        super(Actor, self).__init__()
        self.seed = torch.manual_seed(seed)
        # weight initialization gains 
        self.relu_gain = nn.init.calculate_gain('relu')
        self.tanh_gain = nn.init.calculate_gain('tanh')
        # layers
        self.fc1 = nn.Linear(state_size, fc1_units)
        self.fc2 = nn.Linear(fc1_units, fc2_units)
        self.fc3 = nn.Linear(fc2_units, action_size)
        self.bn1 = nn.BatchNorm1d(fc1_units)
        
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.xavier_uniform_(self.fc1.weight.data, self.relu_gain)
        nn.init.xavier_uniform_(self.fc2.weight.data, self.relu_gain)
        nn.init.xavier_uniform_(self.fc3.weight.data, self.tanh_gain)

    def forward(self, state):
        """Build an actor (policy) network that maps states -> actions."""
        x = self.bn1(self.fc1(state).relu())
        x = self.fc2(x).relu()
        return self.fc3(x).tanh()

In [ ]:
class Critic(nn.Module):
    """Critic (Value) Model."""

    def __init__(self, num_agents, state_size, action_size, seed, 
                 fcs1_units=400, fc2_units=300):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            seed (int): Random seed
            fcs1_units (int): Number of nodes in the first hidden layer
            fc2_units (int): Number of nodes in the second hidden layer
        """
        super(Critic, self).__init__()
        self.seed = torch.manual_seed(seed)
        # weight initialization gains 
        self.relu_gain = nn.init.calculate_gain('relu')
        self.linear_gain = nn.init.calculate_gain('linear')
        # layers
        self.fcs1 = nn.Linear(num_agents*state_size, fcs1_units)
        self.fc2 = nn.Linear(fcs1_units+(num_agents*action_size), fc2_units)
        self.fc3 = nn.Linear(fc2_units, 1)
        self.bn1 = nn.BatchNorm1d(fcs1_units)
        
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.xavier_uniform_(self.fcs1.weight.data, self.relu_gain)
        nn.init.xavier_uniform_(self.fc2.weight.data, self.relu_gain)
        nn.init.xavier_uniform_(self.fc3.weight.data, self.relu_gain)

    def forward(self, state, action):
        """Build a critic (value) network that maps (
           state, action) pairs -> Q-values."""
        xs = self.bn1(self.fcs1(state).relu())
        x = torch.cat((xs, action), dim=1)
        x = self.fc2(x).relu()
        return self.fc3(x)

In [ ]:
class ReplayBuffer:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self, buffer_size, batch_size, seed, device):
        """Initialize a ReplayBuffer object.
        Params
        ======
            buffer_size (int): maximum size of buffer
            batch_size (int): size of each training batch
            seed (int): random seed
            device (torch.device): - (type='cuda', index=0) for GPU 0
                                   - (type='cpu') for CPU
        """
        self.memory = deque(maxlen=buffer_size)  # internal memory (deque)
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", 
                                     field_names=["state", "action", "reward", 
                                                  "next_state", "done"])
        self.seed = random.seed(seed)
        self.device = device
    
    def add(self, states, actions, rewards, next_states, dones):
        """Add a new experience to memory."""
        e = self.experience(states, actions, rewards, next_states, dones)
        self.memory.append(e)
    
    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        experiences = random.sample(self.memory, k=self.batch_size)

        states = torch.from_numpy(np.vstack(
            [e.state for e in experiences if e is not None]
        )).float().to(self.device)
        
        actions = torch.from_numpy(np.vstack(
            [e.action for e in experiences if e is not None]
        )).float().to(self.device)
        
        rewards = torch.from_numpy(np.vstack(
            [e.reward for e in experiences if e is not None]
        )).float().to(self.device)
        
        next_states = torch.from_numpy(np.vstack(
            [e.next_state for e in experiences if e is not None]
        )).float().to(self.device)
        
        dones = torch.from_numpy(np.vstack(
            [e.done for e in experiences if e is not None]
        ).astype(np.uint8)).float().to(self.device)

        return (states, actions, rewards, next_states, dones)

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)

In [ ]:
import copy
class OUNoise:
    """Ornstein-Uhlenbeck process."""

    def __init__(self, size, seed, mu=0., theta=0.15, sigma=0.2):
        """Initialize parameters and noise process."""
        self.mu = mu * np.ones(size)
        self.theta = theta
        self.sigma = sigma
        self.seed = random.seed(seed)
        self.reset()

    def reset(self):
        """Reset the internal state (= noise) to mean (mu)."""
        self.state = copy.copy(self.mu)

    def sample(self):
        """Update internal state and return it as a noise sample."""
        x = self.state
        dx = self.theta * (self.mu - x) + \
             self.sigma * np.array([random.random() for i in range(len(x))])
        self.state = x + dx
        return self.state

In [ ]:
class Agent():
    """Interacts with and learns from the environment."""
    
    def __init__(self, index, num_agents, state_size, action_size, 
                 random_seed, device):
        """Initialize an Agent object.
        
        Params
        ======
            num_agents (int): number of agents
            state_size (int): dimension of each state
            action_size (int): dimension of each action
            random_seed (int): random seed
        """
        self.index = index 
        self.num_agents = num_agents
        self.state_size = state_size
        self.action_size = action_size
        self.random_seed = random_seed
        self.seed = random.seed(random_seed)
        self.device = device

        # Actor Network (w/ Target Network)
        self.actor_local = Actor(self.num_agents, self.state_size, self.action_size, 
                                 self.random_seed).to(self.device)
        self.actor_target = Actor(self.num_agents, self.state_size, self.action_size, 
                                  self.random_seed).to(self.device)
        self.actor_optimizer = optim.Adam(self.actor_local.parameters(), lr=LR_ACTOR)

        # Critic Network (w/ Target Network)
        self.critic_local = Critic(self.num_agents, self.state_size, self.action_size, 
                                   self.random_seed).to(self.device)
        self.critic_target = Critic(self.num_agents, self.state_size, self.action_size, 
                                    self.random_seed).to(self.device)
        self.critic_optimizer = optim.Adam(self.critic_local.parameters(), lr=LR_CRITIC, 
                                           weight_decay=WEIGHT_DECAY)

        # Noise process
        self.noise = OUNoise(self.action_size, self.random_seed, sigma=SIGMA)

    def sample_and_learn(self, agents, shared_memory):
        # Learn, if enough samples are available in memory
        if len(shared_memory) > BATCH_SIZE:
            for _ in range(5):
                experiences = shared_memory.sample()
                self.learn(agents, experiences, GAMMA)

    def act(self, state, add_noise=True):
        """Returns actions for given state as per current policy."""
        state = torch.from_numpy(state).float().to(self.device)
        self.actor_local.eval()
        with torch.no_grad():
            action = self.actor_local(state).cpu().data.numpy()
        self.actor_local.train()
        if add_noise:
            action += self.noise.sample()
        return np.clip(action, -1.0, 1.0)

    def reset(self):
        self.noise.reset()

    def learn(self, agents, experiences, gamma):
        """Update policy and value parameters using given batch of experience tuples.
        Q_targets = r + γ * critic_target(next_state, actor_target(next_state))
        where:
            actor_target(state) -> action
            critic_target(state, action) -> Q-value
        Params
        ======
            experiences (Tuple[torch.Tensor]): tuple of (s, a, r, s', done) tuples 
            gamma (float): discount factor
        """
        states_flat, actions_flat, rewards_flat, next_states_flat, \
            dones_flat = experiences
    
        states = states_flat.reshape(-1, self.num_agents, self.state_size)
        actions = actions_flat.reshape(-1, self.num_agents, self.action_size)
        rewards = rewards_flat.reshape(-1, self.num_agents)
        next_states = next_states_flat.reshape(-1, self.num_agents, 
                                               self.state_size)
        dones = torch.max(dones_flat, dim=1)[0].reshape(-1,1)
        
        # ---------------------------- update critic ---------------------------- #
        # Get predicted next-state actions and Q values from target models
        next_actions = ()
        for i, agent in enumerate(agents):
            next_actions += (agent.actor_target(next_states[:,i]),)
        next_actions_flat = torch.cat(next_actions, dim=1)
        Q_targets_next = self.critic_target(next_states_flat, next_actions_flat)
        # Compute Q targets for current states (y_i)
        Q_targets = rewards[:,self.index].reshape(-1,1) + \
                    (gamma * Q_targets_next * (1 - dones))
        # Compute critic loss
        Q_expected = self.critic_local(states_flat, actions_flat)
        critic_loss = F.mse_loss(Q_expected, Q_targets)
        # Minimize the loss
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        torch.nn.utils.clip_grad_norm_(self.critic_local.parameters(), 1)
        self.critic_optimizer.step()

        # ---------------------------- update actor ---------------------------- #
        # Compute actor loss
        pred_actions = ()
        for i, agent in enumerate(agents):
            pred_actions += (agent.actor_local(states[:,i]),)
        pred_actions_flat = torch.cat(pred_actions, dim=1)
        actor_loss = -self.critic_local(states_flat, pred_actions_flat).mean()
        # Minimize the loss
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()

        # ----------------------- update target networks ----------------------- #
        self.soft_update(self.critic_local, self.critic_target, TAU)
        self.soft_update(self.actor_local, self.actor_target, TAU)                     

    def soft_update(self, local_model, target_model, tau):
        """Soft update model parameters.
        θ_target = τ*θ_local + (1 - τ)*θ_target
        Params
        ======
            local_model: PyTorch model (weights will be copied from)
            target_model: PyTorch model (weights will be copied to)
            tau (float): interpolation parameter 
        """
        for target_param, local_param in zip(target_model.parameters(), 
                                             local_model.parameters()):
            target_param.data.copy_(tau*local_param.data + \
                                    (1.0-tau)*target_param.data)
            

In [ ]:
# check if CUDA is available
use_cuda = torch.cuda.is_available()
print("Use CUDA:",  use_cuda)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
# agent-0, agent-1
import random

agents = [Agent(index=0, num_agents=num_agents, state_size=state_size, 
                action_size=action_size, random_seed=SEED, device=device),
          Agent(index=1, num_agents=num_agents, state_size=state_size, 
                action_size=action_size, random_seed=SEED, device=device),]

In [ ]:
for i, agent in enumerate(agents):
    print("=== Agent %d" % (i+1), "="*64)
    print(agent.actor_local)
    print(agent.actor_target)
    print(agent.actor_optimizer)
    print(agent.critic_local)
    print(agent.critic_target)
    print(agent.critic_optimizer)


In [ ]:
def maddpg(n_episodes, max_t, print_every, random_seed, device):
    # Replay memory
    shared_memory = ReplayBuffer(BUFFER_SIZE, BATCH_SIZE, 
                                 random_seed, device)
    
    acc_scores = []
    acc_scores_window = deque(maxlen=100)  # last 100 scores
    for i_episode in range(1, n_episodes+1):
        # reset the environment
        env_info = env.reset(train_mode=True)[brain_name]
        # get the current state (for each agent)
        states = env_info.vector_observations
        states_flat = np.hstack(states)
        # reset agents
        for agent in agents:
            agent.reset()
        # initialize the score (for each agent)
        max_scores = 0
        actions = np.zeros((num_agents, action_size))
        for t in range(max_t):
            # get action
            for i, agent in enumerate(agents):
                actions[i] = agent.act(states[i].reshape(1,-1), add_noise=True)
            actions_flat = np.hstack(actions)
            # send all actions to tne environment
            env_info = env.step(actions)[brain_name]
            # get next states, rewards and done flags (for each agent)
            next_states = env_info.vector_observations
            next_states_flat = np.hstack(next_states)
            rewards = env_info.rewards
            rewards_flat = np.array(rewards)
            dones = env_info.local_done
            dones_flat = np.array(dones)
            # update shared memory
            shared_memory.add(states_flat, actions_flat, rewards_flat, 
                              next_states_flat, dones_flat)
            # update state
            states = next_states
            states_flat = next_states_flat
            # accumulate rewards
            max_scores += max(rewards)
            # exit loop if episode finished
            if np.any(dones):
                break
        
        # sample and learn
        for agent in agents:
            agent.sample_and_learn(agents, shared_memory)

        # save most recent scores
        acc_scores.append(max_scores)
        acc_scores_window.append(max_scores)
        
        # model checkpoints
        for i, agent in enumerate(agents):
            torch.save(agent.actor_local.state_dict(), "agent%d_actor.pth" % i)
            torch.save(agent.critic_local.state_dict(), "agent%d_critic.pth" % i)
        
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, 
                                                           np.mean(acc_scores_window)), 
              end="")
        if i_episode % print_every == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, 
                                                               np.mean(acc_scores_window)))
        if np.mean(acc_scores_window)>=0.5:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(
                  i_episode-100, np.mean(acc_scores_window)))
            break
            
    return acc_scores

In [ ]:
import time
MAX_EPISODES = 10000

tic = time.time()
scores = maddpg(n_episodes=MAX_EPISODES, max_t=1000, 
                print_every=1000, random_seed=SEED, device=device)
toc = time.time()
print()
print("Runtime (in seconds) %d" % int(toc-tic))

In [ ]:
import matplotlib
print("matplotlib %s" % matplotlib.__version__)
import matplotlib.pyplot as plt

# Set plotting options
%matplotlib inline

In [ ]:
scores = np.array(scores)
scores_window = [np.mean(scores[max(0,i-100):i+1]) 
                 for i in range(len(scores))]

# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores, label="window size 1")
plt.plot(np.arange(1, len(scores)+1), scores_window, label="window size 100")
plt.plot(np.arange(1, len(scores)+1), [0.5]*len(scores), 
         linestyle='--', color='red')
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.legend()
plt.show()